In [3]:
import tensorflow_datasets as tfd
import tensorflow as tf
import matplotlib.pyplot as plt
import segmentation_models as sb

tf.config.set_visible_devices([], 'GPU')

Segmentation Models: using `keras` framework.


In [49]:
# the network must OUTPUT in logits [-inf, inf]
def Jindex(target, pred, smooth=1e-6):
    intersection = tf.reduce_sum(target * pred)
    union = tf.reduce_sum(target + pred) - intersection

    return (intersection + smooth) / (union + smooth)


# # I hypothesis from_logits is the cause of NaN
def loss_fn(target, sample, weight, alpha=1, beta=0.5):
    loss = alpha * tf.reduce_mean(
        tf.keras.losses.categorical_crossentropy(target, sample),
    )
    return loss + beta * (1 - Jindex(target, sample, weight))


one = tf.constant([0.5, 0, 0, 0], tf.float32)
two = tf.constant([0.5, 0, 0, 0], tf.float32)

Jindex(one[tf.newaxis, :], two[tf.newaxis, :])


<tf.Tensor: shape=(), dtype=float32, numpy=0.33333424>

In [ ]:
ds = tfd.load('cats_vs_dogs', split='train', shuffle_files=True)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=[150, 150, 3])
base_model.trainable = False

inputs = tf.keras.Input([150, 150, 3])
x = base_model(inputs, training=False)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
def process(x):
    image = x['image']
    label = x['label']
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    image = tf.image.resize(image, [150, 150])

    return image, label


In [16]:
metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.18157327>

In [20]:
metric.reset_state()


In [48]:
y_true = [[1], [1], [0], [0]]
y_pred = [[1], [1], [1], [0]]
tf.reduce_mean(tf.keras.metrics.binary_accuracy(y_true, y_pred))

<tf.Tensor: shape=(), dtype=float32, numpy=0.75>

In [54]:
total_acc / total_num

<tf.Tensor: shape=(), dtype=float32, numpy=0.75>

(<tf.Tensor: shape=(8, 1), dtype=int64, numpy=
 array([[1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0]])>,
 <tf.Tensor: shape=(8, 1), dtype=int64, numpy=
 array([[1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0]])>)

In [76]:
metric = tf.keras.metrics.BinaryAccuracy()
# for inputs, targets in ds.map(process).batch(8).take(1):
#     predictions = model(inputs)
#     metric.update_state(predictions, targets)
# print(metric.result().numpy())
# metric.reset_state()
total_acc = 0
total_num = 0

for inputs, targets in ds.map(process).batch(8):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss_value = loss_fn(targets, predictions)

    total_num += 1
    total_acc += tf.reduce_mean(tf.keras.metrics.binary_accuracy(tf.reshape(targets, [-1, 1]), tf.cast(predictions, tf.int64)))

    # print(tf.reduce_mean(tf.keras.metrics.binary_accuracy(targets, predictions)))
    # print(total_acc / total_num)
    print(tf.reduce_mean(tf.keras.metrics.binary_accuracy(tf.reshape(targets, [-1, 1]), tf.cast(predictions, tf.int64))))


    grad = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grad, model.trainable_weights))
metric.reset_state()

# model.compile('adam', loss_fn, ['acc'])
# model.fit(ds.map(process).batch(8), epochs=1)

tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.75, shape=(), dtype=float32)
tf.Tensor(0.875, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), 

KeyboardInterrupt: 

In [ ]:
metric = tf.keras.metrics.BinaryAccuracy()
for inputs, targets in ds.map(process).batch(8).take(1):
    predictions = model(inputs)
    metric.update_state(predictions, targets)
print(metric.result().numpy())
metric.reset_state()


# for inputs, targets in ds.map(process).batch(8):
#     with tf.GradientTape() as tape:
#         predictions = model(inputs)
#         loss_value = loss_fn(targets, predictions)

#     metric.update_state(predictions, targets)

#     print(loss_value, metric.result().numpy())
#     grad = tape.gradient(loss_value, model.trainable_weights)
#     optimizer.apply_gradients(zip(grad, model.trainable_weights))

# metric.reset_state()
